## Deliverable 2 : Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
import sys
sys.path.append("/Users/jane.mathew/Desktop")
from api_keys import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,65.25,83,57,1.01,broken clouds
1,1,Rawson,AR,-43.3002,-65.1023,66.16,57,12,10.47,few clouds
2,2,Madaba,JO,31.5833,35.6667,48.88,59,59,6.38,broken clouds
3,3,Talnakh,RU,69.4865,88.3972,-5.80,97,100,3.22,light snow
4,4,Mount Gambier,AU,-37.8333,140.7667,76.64,42,47,7.20,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Mount Gambier,AU,-37.8333,140.7667,76.64,42,47,7.20,scattered clouds
18,18,Bonthe,SL,7.5264,-12.5050,79.79,84,11,7.81,few clouds
21,21,Moron,AR,-34.6534,-58.6198,82.29,66,41,1.99,scattered clouds
27,27,Avarua,CK,-21.2078,-159.7750,84.25,70,75,16.11,broken clouds
28,28,Kavieng,PG,-2.5744,150.7967,83.55,69,100,7.38,overcast clouds
31,31,Vaini,TO,-21.2000,-175.2000,78.96,94,90,9.22,light intensity shower rain
32,32,Butaritari,KI,3.0707,172.7902,81.46,77,99,17.45,overcast clouds
40,40,Carnarvon,AU,-24.8667,113.6333,78.87,36,0,17.27,clear sky
50,50,Rikitea,PF,-23.1203,-134.9692,78.10,79,100,19.91,overcast clouds
51,51,Lorengau,PG,-2.0226,147.2712,83.14,72,81,8.52,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                184
City                   184
Country                184
Lat                    184
Lng                    184
Max Temp               184
Humidity               184
Cloudiness             184
Wind Speed             184
Current Description    184
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_new = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_new[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Mount Gambier,AU,76.64,scattered clouds,-37.8333,140.7667,
18,Bonthe,SL,79.79,few clouds,7.5264,-12.5050,
21,Moron,AR,82.29,scattered clouds,-34.6534,-58.6198,
27,Avarua,CK,84.25,broken clouds,-21.2078,-159.7750,
28,Kavieng,PG,83.55,overcast clouds,-2.5744,150.7967,
31,Vaini,TO,78.96,light intensity shower rain,-21.2000,-175.2000,
32,Butaritari,KI,81.46,overcast clouds,3.0707,172.7902,
40,Carnarvon,AU,78.87,clear sky,-24.8667,113.6333,
50,Rikitea,PF,78.10,overcast clouds,-23.1203,-134.9692,
51,Lorengau,PG,83.14,broken clouds,-2.0226,147.2712,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Mount Gambier,AU,76.64,scattered clouds,-37.8333,140.7667,The Commodore
18,Bonthe,SL,79.79,few clouds,7.5264,-12.5050,Bonthe Holiday Village
21,Moron,AR,82.29,scattered clouds,-34.6534,-58.6198,Hotel Morón
27,Avarua,CK,84.25,broken clouds,-21.2078,-159.7750,Paradise Inn
28,Kavieng,PG,83.55,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
31,Vaini,TO,78.96,light intensity shower rain,-21.2000,-175.2000,Keleti Beach Resort
32,Butaritari,KI,81.46,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
40,Carnarvon,AU,78.87,clear sky,-24.8667,113.6333,Hospitality Carnarvon
50,Rikitea,PF,78.10,overcast clouds,-23.1203,-134.9692,People ThankYou
51,Lorengau,PG,83.14,broken clouds,-2.0226,147.2712,Lorengau Harbourside Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0,31.0),zoom_level=1)
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

In [13]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))